*Author: C Mitchell*

**This notebook deals with the TNC Basin, Pemaquid and Bigelow data.**

The purpose of this notebook is to create a new sample list which includes the scaled biomass numbers that Dave put together. We needed to scale the biomass numbers because the sensor didn't view the full sample, only a part of it.

To do this - we calculated 
1) the volume of the segment of sensor-field-of-view-cone that contained seaweed
2) the total volume of the seaweed sample (assuming a cylinder)
3) the percentage of the total volume sensed
4) the wet and dry weights scaled by the percentage of the total volume sensed

**The output of this notebook is `sample_list_v3.csv`**

# Initializing workflow

Importing modules and the data

In [1]:
import pandas as pd
import os
import re
import datetime as dt

The scaled biomass data:

In [5]:
scaledinfo = pd.read_excel('X:/projects/meifsci-seaweed-drones/analysis/data/sample_list_with_areas_v2.xlsx')

In [6]:
scaledinfo

,idkey,Site,Sample,Wet_weight,Dry_weight,Canopy_Depth,hours,minutes,seconds,n,...,Viewing half-angle,Sensor diameter (cm) @ card,Sensor diameter (cm) @ canopy top,Canopy diameter (cm),% area sensed,Cone volume @ Canopy depth (cm3),Total volume (cm3),% volume sensed,wet weight sensed,dry weight sensed
0,Basin_Asco1b,Basin,Asco1b,39.0,13.3,1.0,13.0,51.0,31.0,40.0,...,11.5,10.986424,10.579520,20.5,26.633250,91.331031,330.063578,27.670739,10.791588,3.680208
1,Basin_Asco1c,Basin,Asco1c,39.0,13.3,1.0,15.0,19.0,45.0,40.0,...,11.5,10.986424,10.579520,20.5,26.633250,91.331031,330.063578,27.670739,10.791588,3.680208
2,Basin_Asco1,Basin,Asco1,39.0,13.3,1.0,10.0,53.0,31.0,40.0,...,11.5,10.986424,10.579520,20.5,26.633250,91.331031,330.063578,27.670739,10.791588,3.680208
3,Basin_Asco2b,Basin,Asco2b,83.3,28.7,2.8,13.0,58.0,50.0,40.0,...,11.5,10.986424,9.847091,23.0,18.329907,238.861291,1163.331760,20.532517,17.103587,5.892832
4,Basin_Asco2c,Basin,Asco2c,83.3,28.7,2.8,15.0,27.0,48.0,40.0,...,11.5,10.986424,9.847091,23.0,18.329907,238.861291,1163.331760,20.532517,17.103587,5.892832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Bigelow_Fucus5,Bigelow,Fucus5,542.8,NaN,10.5,NaN,NaN,NaN,NaN,...,4.0,10.908583,9.440120,31.0,9.273242,855.157037,7925.060168,10.790543,58.571068,NaN
95,Bigelow_Fucus6,Bigelow,Fucus6,591.2,NaN,14.5,NaN,NaN,NaN,NaN,...,4.0,10.908583,8.880705,31.5,7.948292,1118.859916,11300.014251,9.901403,58.537093,NaN
96,Bigelow_Fucus7,Bigelow,Fucus7,655.3,NaN,14.6,NaN,NaN,NaN,NaN,...,4.0,10.908583,8.866720,31.5,7.923277,1125.044361,11377.945383,9.887940,64.795668,NaN
97,Bigelow_Fucus8,Bigelow,Fucus8,698.8,NaN,14.8,NaN,NaN,NaN,NaN,...,4.0,10.908583,8.838749,33.0,7.173874,1137.354844,12658.419279,8.984967,62.786952,NaN


# Data wrangling

We have all the data, we just want to simplify it a bit

In [15]:
varskeep = ['idkey','Site','Sample','Wet_weight','Dry_weight','Canopy_Depth',
            'hours','minutes','seconds','n',
           'wet weight sensed','dry weight sensed']

scaleddf = scaledinfo[varskeep].set_index('idkey').rename(columns={'wet weight sensed' : 'wet_weight_sensed',
                                                               'dry weight sensed' : 'dry_weight_sensed'})

In [16]:
scaleddf

,Site,Sample,Wet_weight,Dry_weight,Canopy_Depth,hours,minutes,seconds,n,wet_weight_sensed,dry_weight_sensed
idkey,,,,,,,,,,,
Basin_Asco1b,Basin,Asco1b,39.0,13.3,1.0,13.0,51.0,31.0,40.0,10.791588,3.680208
Basin_Asco1c,Basin,Asco1c,39.0,13.3,1.0,15.0,19.0,45.0,40.0,10.791588,3.680208
Basin_Asco1,Basin,Asco1,39.0,13.3,1.0,10.0,53.0,31.0,40.0,10.791588,3.680208
Basin_Asco2b,Basin,Asco2b,83.3,28.7,2.8,13.0,58.0,50.0,40.0,17.103587,5.892832
Basin_Asco2c,Basin,Asco2c,83.3,28.7,2.8,15.0,27.0,48.0,40.0,17.103587,5.892832
...,...,...,...,...,...,...,...,...,...,...,...
Bigelow_Fucus5,Bigelow,Fucus5,542.8,NaN,10.5,NaN,NaN,NaN,NaN,58.571068,NaN
Bigelow_Fucus6,Bigelow,Fucus6,591.2,NaN,14.5,NaN,NaN,NaN,NaN,58.537093,NaN
Bigelow_Fucus7,Bigelow,Fucus7,655.3,NaN,14.6,NaN,NaN,NaN,NaN,64.795668,NaN


In [17]:
scaleddf.to_csv('../data/sample_list_v3.csv')